# Yolo5 학습하기
##1. 구글 드라이브 연동

In [ ]:
# prompt: 구글 드라이브랑 연결

from google.colab import drive
drive.flush_and_unmount()  # 기존 마운트 해제
drive.mount('/content/drive')  # 다시 마운트

In [ ]:
%cd /content/drive/MyDrive

In [ ]:
%pwd

# 2. Yolov5 설치

In [ ]:
# 기존에 설치를 완료한 경우에는 해당 경로로 이동만 하면 됩니다.
%cd /content/drive/MyDrive/yolov5

In [ ]:
##
#clone YOLOv5 and
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt # install dependencies

In [ ]:
!pip install Pillow==10.3

#

# 3. 이미지 파일 관리

In [ ]:
!mkdir -p Train/labels
!mkdir -p Train/images
!mkdir -p Val/labels
!mkdir -p Val/images


In [ ]:
##검증 데이터 만들기
import os
import shutil
from sklearn.model_selection import train_test_split

def create_validation_set(train_path, val_path, split_ratio=0.3):
    """
    Train 데이터의 일부를 Val로 이동
    """
    # 필요한 디렉토리 생성
    os.makedirs(os.path.join(val_path, 'images'), exist_ok=True)
    os.makedirs(os.path.join(val_path, 'labels'), exist_ok=True)

    # Train 이미지 리스트 가져오기
    train_images = os.listdir(os.path.join(train_path, 'images'))
    train_images = [f for f in train_images if f.endswith(('.jpg', '.jpeg', '.png'))]

    # Train/Val 분할
    _, val_images = train_test_split(train_images,
                                   test_size=split_ratio,
                                   random_state=42)

    # Val로 파일 복사
    for image_file in val_images:
        # 이미지 복사
        src_image = os.path.join(train_path, 'images', image_file)
        dst_image = os.path.join(val_path, 'images', image_file)
        shutil.copy2(src_image, dst_image)

        # 라벨 파일 복사
        label_file = os.path.splitext(image_file)[0] + '.txt'
        src_label = os.path.join(train_path, 'labels', label_file)
        dst_label = os.path.join(val_path, 'labels', label_file)
        if os.path.exists(src_label):
            shutil.copy2(src_label, dst_label)

    print(f"Created validation set with {len(val_images)} images")

# 실행
train_path = '/content/drive/MyDrive/yolov5/Train'
val_path = '/content/drive/MyDrive/yolov5/Val'

create_validation_set(train_path, val_path)

In [ ]:
def check_dataset():
    train_path = '/content/drive/MyDrive/yolov5/Train'
    val_path = '/content/drive/MyDrive/yolov5/Val'

    # Train 데이터 확인
    train_images = len(os.listdir(os.path.join(train_path, 'images')))
    train_labels = len(os.listdir(os.path.join(train_path, 'labels')))

    # Val 데이터 확인
    val_images = len(os.listdir(os.path.join(val_path, 'images')))
    val_labels = len(os.listdir(os.path.join(val_path, 'labels')))

    print("Dataset status:")
    print(f"Train - Images: {train_images}, Labels: {train_labels}")
    print(f"Val - Images: {val_images}, Labels: {val_labels}")

# 데이터셋 상태 확인
check_dataset()

# 4. 학습시작

In [ ]:
#필요 라이브러리 임포트하기
import torch
import os
from IPython.display import Image, clear_output  # to display images

In [ ]:
%pwd

In [ ]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt

def _preproc(image, output_height=640, output_width=640, resize_side=640):
    h, w = image.shape[0], image.shape[1]
    scale = tf.cond(tf.less(h, w), lambda: resize_side / h, lambda: resize_side / w)
    resized_image = tf.compat.v1.image.resize_bilinear(tf.expand_dims(image, 0), [int(h * scale), int(w * scale)])
    cropped_image = tf.compat.v1.image.resize_with_crop_or_pad(resized_image, output_height, output_width)
    return tf.squeeze(cropped_image)

def Create_npy(imagespath, imgsize, ext):
    images_list = [img_name for img_name in os.listdir(imagespath) if
                   os.path.splitext(img_name)[1].lower() == '.' + ext.lower()]
    calib_dataset = np.zeros((len(images_list), imgsize, imgsize, 3), dtype=np.float32)

    writer = SummaryWriter(log_dir='/content/drive/MyDrive/yolov5/runs/preprocessed_images')

    for idx, img_name in enumerate(sorted(images_list)):
        img_path = os.path.join(imagespath, img_name)
        try:
            if os.path.getsize(img_path) == 0:
                print(f"Error: {img_path} is empty.")
                continue

            img = Image.open(img_path)
            img = img.convert("RGB")
            img_np = np.array(img)

            img_preproc = _preproc(img_np, imgsize, imgsize, imgsize)
            calib_dataset[idx, :, :, :] = img_preproc.numpy().astype(np.uint8)

            # 이미지 시각화
            plt.imshow(img_preproc.numpy().astype(np.uint8))
            plt.title(f"Preprocessed Image: {img_name}")
            plt.axis("off")

            # TensorBoard에 기록
            fig = plt.gcf()
            writer.add_figure(f"Preprocessed Image {idx}", fig)
            plt.close(fig)

            print(f"Processed image {img_path}")

        except Exception as e:
            print(f"Error processing image {img_path}: {e}")

    writer.close()
    np.save('/content/drive/MyDrive/yolov5/runs/preprocessed_images/calib_set.npy', calib_dataset)

# 실행 예시
Create_npy('/content/drive/MyDrive/yolov5/Train/images', 640, 'png')


In [ ]:
# "cannot identify image file" 에러가 발생하는 경우, PILLOW Version을 "!pip install Pillow==10.1" 로 변경하여 설치합니다.
Create_npy('/content/drive/MyDrive/yolov5/Train/images', 640, 'png')

In [ ]:
!pip uninstall -y wandb

In [ ]:
#모델 학습하기
!python  /content/drive/MyDrive/yolov5/train.py  --img 640 --batch 16 --epochs 300 --data /content/drive/MyDrive/yolov5/data.yaml --weights yolov5n.pt --cache --patience 10  # 예: 10 에포크 동안 개선이 없으면 중단


In [ ]:
!pip install --upgrade Pillow


In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from torch.utils.tensorboard import SummaryWriter
from matplotlib.patches import Rectangle

def visualize_class_objects(image, boxes, class_labels, img_size=(640, 640)):
    """
    이미지에서 바운딩 박스를 그려서 클래스별 객체를 시각화합니다.

    :param image: 원본 이미지 (numpy 배열)
    :param boxes: 바운딩 박스 좌표 (각 클래스별로)
    :param class_labels: 클래스 레이블 (배열 형태)
    :param img_size: 출력 이미지 크기
    :return: 시각화된 이미지
    """
    plt.figure(figsize=(10, 10))
    plt.imshow(image)

    for i, box in enumerate(boxes):
        y_min, x_min, y_max, x_max = box
        class_label = class_labels[i]
        plt.gca().add_patch(Rectangle((x_min, y_min), x_max - x_min, y_max - y_min,
                                      linewidth=2, edgecolor='r', facecolor='none'))
        plt.text(x_min, y_min, str(class_label), color='r', fontsize=12)

    plt.axis('off')
    plt.close()

    return np.array(plt.gcf())

# YOLOv5 모델 불러오기
def load_model(model_path='yolov5s.pt'):
    model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path)
    return model

# 예측 및 시각화
def predict_and_visualize(model, image_path, img_size=(512, 512)):
    img = Image.open(image_path).convert("RGB")
    img_np = np.array(img)

    # 모델 예측
    results = model(img)
    boxes = results.xyxy[0].numpy()  # 바운딩 박스 좌표 (x_min, y_min, x_max, y_max)
    class_labels = results.names  # 클래스 이름

    # 클래스 레이블 및 바운딩 박스를 이미지에 시각화
    visualized_image = visualize_class_objects(img_np, boxes, class_labels)

    # TensorBoard에 기록
    writer = SummaryWriter(log_dir='/content/drive/MyDrive/yolov5/runs/inference')
    writer.add_image('Predicted Image', visualized_image, global_step=0)
    writer.close()

    # 시각화된 이미지 반환
    plt.imshow(visualized_image)
    plt.title("Predicted Image")
    plt.axis("off")
    plt.show()

# 모델 불러오기
model = load_model('/content/drive/MyDrive/yolov5/runs/train/exp6/weights/best.pt')  # 학습한 모델 파일 경로

# 예측 및 시각화 실행
predict_and_visualize(model, '/content/drive/MyDrive/yolov5/Test/image.png')


# 5. yolov5모델 학습 결과 검증

In [ ]:
!pkill -f "tensorboard"  # 모든 TensorBoard 프로세스를 종료

In [ ]:
# Start tensorboard
# Launch after you have started training
# logs save in the folder "runs"
%load_ext tensorboard
%tensorboard --logdir /content/drive/MyDrive/yolov5/runs/train/exp7

In [ ]:
!python /content/drive/MyDrive/yolov5/detect.py --weights /content/drive/MyDrive/yolov5/runs/train/exp4/weights/best.pt --img 640 --conf 0.1 --source /content/drive/MyDrive/yolov5/Train/images

In [ ]:
!pip install ultralytics

In [ ]:
!python /content/drive/MyDrive/yolov5/detect.py --weights /content/drive/MyDrive/yolov5/runs/train/exp4/weights/best.pt --img 640 --conf 0.1 --source /content/drive/MyDrive/yolov5/Train/숏.mp4

In [ ]:
#display inference on ALL test images

import glob
from IPython.display import Image, display

for imageName in glob.glob('/content/drive/MyDrive/yolov5/yolov5/runs/detect/exp11/*.png')[:10]: #이미지 파일 형식에 맞춰 .png 또는 .jpg 등으로 수정
    display(Image(filename=imageName))
    print("\n")